In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import svm

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet 
from nltk.tokenize import word_tokenize


### Input Dataset

In [2]:
df_train = pd.read_csv("data_train_clean.csv")
df_test = pd.read_csv("data_test_clean.csv")

In [3]:
df_train.head()

,Tweet_Parsed,HS
0,kadang will be kind enough to show you how to...,0
1,ternyata komunis juga bisa menangis,0
2,user user kenapa harus bom seperti benar men...,0
3,user sumpah kaya kalau habis iya dilanjut olim...,0
4,rt user user user user user fungsi media sosia...,1


In [4]:
df_test.head()

,Tweet_Parsed,HS
0,ganteng tapi berengsek buat apa who do you thi...,1
1,sakit jiwa kali,0
2,does insulting one for being cina kafir make y...,0
3,user user karena partaimu partai demokrasi ind...,1
4,user user jangan salah itu janji jokowi di tu...,1


### Split Data

In [4]:
X_train = df_train['Tweet_Parsed']
y_train = df_train['HS']

X_test = df_test['Tweet_Parsed']
y_test = df_test['HS']

In [5]:
X_train.head()

0     kadang will be kind enough to show you how to...
1                 ternyata komunis juga bisa menangis 
2    user user kenapa harus bom  seperti  benar men...
3    user sumpah kaya kalau habis iya dilanjut olim...
4    rt user user user user user fungsi media sosia...
Name: Tweet_Parsed, dtype: object

### Model FastText

In [6]:
tokenized_corpus = [word_tokenize(sentence) for sentence in df_train['Tweet_Parsed']]

In [7]:
import gensim
from gensim.models import FastText

In [8]:
model = FastText(sentences=tokenized_corpus, vector_size=80, window=5, sg=1, min_count=1)

# Training the FastText model
model.train(tokenized_corpus, total_examples=len(tokenized_corpus), epochs=80)

(17451590, 19916480)

In [9]:
model.wv.most_similar("anda",topn=20)

[('wanda', 0.8061860799789429),
 ('randa', 0.7814821004867554),
 ('janda', 0.7674365043640137),
 ('rwanda', 0.7299870252609253),
 ('pertanda', 0.7178671956062317),
 ('ganda', 0.713076114654541),
 ('dinda', 0.6985489726066589),
 ('andalan', 0.6822708249092102),
 ('belanda', 0.6815142631530762),
 ('malaria', 0.6804636120796204),
 ('ditunda', 0.6786718368530273),
 ('banda', 0.6786677241325378),
 ('baginda', 0.6780444383621216),
 ('poligami', 0.6733474731445312),
 ('dilanda', 0.671912670135498),
 ('tetanda', 0.6682868003845215),
 ('kegagalan', 0.6634942889213562),
 ('tertunda', 0.6619707942008972),
 ('tunda', 0.6594136953353882),
 ('kalap', 0.6592538952827454)]

### Feature Extraction (TD-IDF)

In [10]:
vectorizer = TfidfVectorizer()
train_tfidf= vectorizer.fit_transform(X_train)

In [11]:
TFIDF_train=pd.DataFrame(train_tfidf.toarray(),columns=vectorizer.get_feature_names_out())
TFIDF_train.head()

,aaaaaaah,aaaamiiiiiiinnnn,aaid,aamiin,aamiinkan,aarman,aaron,aarze,ab,aba,...,zona,zone,zoom,zorn,zouk,zuck,zul,zulkifli,zumi,zzed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
test_tfidf = vectorizer.transform(X_test)
TFIDF_test=pd.DataFrame(test_tfidf.toarray(),columns=vectorizer.get_feature_names_out())
TFIDF_test.head()

,aaaaaaah,aaaamiiiiiiinnnn,aaid,aamiin,aamiinkan,aarman,aaron,aarze,ab,aba,...,zona,zone,zoom,zorn,zouk,zuck,zul,zulkifli,zumi,zzed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Feature Expansion (TF-IDF + FastText)

In [13]:
def feature_expansion(df, feature, n):
    for col in tqdm(df.columns): #loop per kolom / feature
        try: 
            sim_word = model.wv.most_similar(col, topn=n)#mencari Similarity untuk feature
        except:
            sim_word = []
        if sim_word != []: #kalo similarity-nya tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua feature yang mempunyai nilai = 0, tetapi mempunyai nilai != 0 pada term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

#### Combine on TF-IDF Data

In [14]:
#Get Features Name
feature_tfidf = vectorizer.get_feature_names_out()

#implement Feature Expansion for TF-IDF Data
df_fe_tfidf_train = feature_expansion(TFIDF_train, feature_tfidf, 10)
df_fe_tfidf_test = feature_expansion(TFIDF_test, feature_tfidf, 10)

100%|████████████████████████████████████████████████████████████████████████████| 22459/22459 [18:28<00:00, 20.25it/s]


In [15]:
df_fe_tfidf_train

,aaaaaaah,aaaamiiiiiiinnnn,aaid,aamiin,aamiinkan,aarman,aaron,aarze,ab,aba,...,zona,zone,zoom,zorn,zouk,zuck,zul,zulkifli,zumi,zzed
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.187829,0.165860,0.195937,0.187829,0.165860,0.187829,0.165860,0.165860,0.187829,0.187829
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.375147,0.375147,0.375147,0.375147,0.375147,0.375147,0.375147,0.375147,0.375147,0.375147
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.131352,0.177675,0.177675,0.287272,0.287272,0.131352,0.260760,0.177675,0.131352,0.131352
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.084048,0.125979,0.227881,0.439753,0.439753,0.439753,0.276052,0.125979,0.084048,0.084048
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.368140,0.364282,0.249555,0.249555,0.249555,0.368140,0.249555,0.249555,0.364282,0.368140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.193957,0.193957,0.193957,0.213188,0.193957,0.183698,0.193957,0.193957,0.193957,0.188253
13240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.520821,0.710663,0.710663,0.710663,0.710663,0.520821,0.710663,0.520821,0.710663,0.520821
13241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.150331,0.150331,0.150331,0.150331,0.150331,0.150331,0.156509,0.150331,0.150331,0.150331
13242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.148830,0.188153,0.148830,0.148830,0.188153,0.148830,0.188153,0.188153,0.148830,0.148830


### Classification (Linear SVC)

In [16]:
svm_class = svm.LinearSVC(random_state=42)
svm_class.fit(df_fe_tfidf_train, y_train)

C:\Users\gilan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\gilan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

#### Testing

In [17]:
test_svm_class=svm_class.predict(df_fe_tfidf_test)

In [18]:
print('\nClassification Report\n')
print(classification_report(y_test, test_svm_class, target_names=['0','1']))


Classification Report

              precision    recall  f1-score   support

           0       0.76      0.73      0.75      2204
           1       0.62      0.66      0.64      1476

    accuracy                           0.70      3680
   macro avg       0.69      0.70      0.69      3680
weighted avg       0.71      0.70      0.70      3680

